In [ ]:
from FuncionAzul import *

In [16]:
java_home = os.environ.get('JAVA_HOME', None)
java_home

'JAVA_HOME="/lib/jvm/java-11-openjdk-amd64/bin/java'

In [ ]:
!echo $SPARK_HOME

In [11]:
os.cpu_count()

12

In [1]:
import os
java_path = "/usr/lib/jvm/java-11-openjdk-amd64/"
java_home = os.environ.get('JAVA_HOME', None)

print("Existing Java Home: ",java_home)

if (not java_home) or (java_path not in java_home):
    os.environ['JAVA_HOME'] = java_path

print("Updated Java Home: ",os.environ.get('JAVA_HOME', None))

Existing Java Home:  JAVA_HOME="/lib/jvm/java-11-openjdk-amd64/bin/java
Updated Java Home:  /usr/lib/jvm/java-11-openjdk-amd64/


In [15]:
from function2 import *
import pickle
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window as W

# Prueba 

In [3]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_SOLICITUD_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_INVEST_UO_FIL_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

NameError: name 'FuncionAzul' is not defined

In [18]:
ni = pd.read_csv('DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new.csv', sep = ';')

In [19]:
ni[:20000].to_csv('DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new_short.csv', sep = ';', index = False)

In [2]:
Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_SOLICITUD_ESP.csv" 
Ruta_Nombre_Input_investigadores = "DATA_SIC_INVEST_UO_FIL_PB_J_MAESTRO_short.csv"     
Ruta_Nombre_Input_relaciones = "DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new_short.csv"
Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx' 
Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv" 
Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones"
Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades" 
Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente" 
Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3"
Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores" 
Ruta_Nombre_Output_relaciones = "test/Nombre_Output_relaciones"
Ruta_Nombre_Output_entidades = "test/Nombre_Output_entidades" 
flag_invente_dir3 = True 
Flag_csv = True

In [3]:
spark = SparkSession.builder.appName('FuncionAzul').config("spark.driver.memory","50G")\
                         .config("spark.executor.memory","50G")\
                         .config("spark.executor.cores","12")\
                         .config("spark.executor.instances","5")\
                         .config("spark.driver.maxResultSize", '128g')\
                         .config("spark.memory.offHeap.enabled", 'true')\
                         .config("spark.memory.offHeap.size", '70g')\
                         .enableHiveSupport().getOrCreate()

22/08/28 04:26:07 WARN Utils: Your hostname, gaszsantana-G7-7500 resolves to a loopback address: 127.0.1.1; using 192.168.8.101 instead (on interface wlp0s20f3)
22/08/28 04:26:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/28 04:26:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# =============================================================================
# Creamos 3 archivos: 
# 1. Nombre_Intermedio_cruce1_entidades.
# 2. Nombre_Output_investigadores.  
# 3. Nombre_Intermedio_cruce1_relaciones.
# =============================================================================
Nombre_Input_entidades = spark.read.csv(Ruta_Nombre_Input_entidades, sep=";", header=True)
Nombre_Input_relaciones = spark.read.csv(Ruta_Nombre_Input_relaciones, sep=";", header=True)
Nombre_Input_relaciones = Nombre_Input_relaciones.drop("Entidad_NIVEL_0", "Entidad_Norm_NIVEL_0", "CIF_NIVEL_0", "Provincia_Entidad_NIVEL_0", "Entidad_NIVEL_1",
                                                      "Entidad_Norm_NIVEL_1", "CIF_NIVEL_1", "Provincia_Entidad_NIVEL_1")
Nombre_Input_investigadores = spark.read.csv(Ruta_Nombre_Input_investigadores, sep=";", header=True)


In [5]:
cruce1 = Nombre_Input_investigadores.\
                                    join(Nombre_Input_relaciones,
                                         Nombre_Input_investigadores.Id ==  Nombre_Input_relaciones.Id_NIVEL_0,"inner")\
                                    .dropDuplicates(Nombre_Input_investigadores.columns)\
                                    .filter("Id_NIVEL_0 is not null")



cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm )
                          ) 

In [6]:
# # Nombre_Intermedio_cruce1_entidades

# Valor maximo del index del archivo Nombre_Input_entidades
maxId = Nombre_Input_entidades.withColumn("Id", Nombre_Input_entidades.Id.cast('int')).select("Id").rdd.max()[0]




data = cruce1.filter("Similitud <0.875")\
             .withColumn("CIF",col("CIF"))\
             .withColumn("Entidad",col("Entidad"))\
             .withColumn("Entidad_Norm",col("Entidad_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("ID_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("NIF_COD",lit(None).cast(StringType()))\
             .withColumn("ACRONIMO",lit(None).cast(StringType()))\
             .withColumn("NOMBRE_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("Nombre_Entidad_Mostrar",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N1_1",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N2_1",lit(None).cast(StringType()))\
             .withColumn("DIRECCION_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("COD_CCAA",lit(None).cast(StringType()))\
             .withColumn("CCAA",lit(None).cast(StringType()))\
             .withColumn("ENLACE_WEB",lit(None).cast(StringType()))\
             .withColumn("SOMMA",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_REGIONAL",lit(None).cast(StringType()))\
             .withColumn("ESTADO_x",lit(None).cast(StringType()))\
             .withColumn("CodigoInvente",lit(None).cast(StringType()))\
             .withColumn("DenominacionSocial",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Codigo",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Descripcion",lit(None).cast(StringType()))\
             .withColumn("NIF",lit(None).cast(StringType()))\
             .withColumn("codigoDir3",lit(None).cast(StringType()))\
             .withColumn("codigoOrigen",lit(None).cast(StringType()))\
             .withColumn("Provincia_Codigo",lit(None).cast(StringType()))\
             .withColumn("C_ID_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_ID_NIVEL_ADMON",lit(None).cast(StringType()))\
             .withColumn("C_ID_TIPO_ENT_PUBLICA",lit(None).cast(StringType()))\
             .withColumn("N_NIVEL_JERARQUICO",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("B_SW_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_ESTADO",lit(None).cast(StringType()))\
             .withColumn("D_VIG_ALTA_OFICIAL",lit(None).cast(StringType()))\
             .withColumn("NIF_CIF",lit(None).cast(StringType()))\
             .withColumn("C_ID_AMB_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("C_DESC_PROV",lit(None).cast(StringType()))\
             .withColumn("CONTACTOS",lit(None).cast(StringType()))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))

# =============================================================================
# Procesamiento para iniciar el archivo deseado a partter del maxId
# =============================================================================
data = data.dropDuplicates(["Entidad_Norm", "Provincia_Entidad"])
data = data.withColumnRenamed("Id", "IdOld")


In [ ]:
cruce1.toPandas().to_csv('test/cruce1.csv', decimal = ',', index = False)

In [7]:
data_count = data.count()

In [8]:
data_count

9

In [9]:
Index = [maxId + i for i in range(1, data_count+1)]
Index = spark.createDataFrame(Index, IntegerType())
Index = Index.withColumnRenamed("value", "Id")




windowSpec = W.orderBy("Dummy")

Index = Index.withColumn("Dummy", monotonically_increasing_id())
Index = Index.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.withColumn("Dummy", monotonically_increasing_id())
data = data.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.join(Index, "Dummy", "outer").drop("Dummy")



Nombre_Intermedio_cruce1_entidades = data.select(Nombre_Input_entidades.columns)\
                                         .withColumn("Cruce", lit(Ruta_Nombre_Input_investigadores+' / '+Ruta_Nombre_Input_relaciones).cast(StringType()))





# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Intermedio_cruce1_entidades, Flag_csv, Ruta_Nombre_Intermedio_cruce1_entidades)


22/08/28 04:26:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/28 04:26:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 0

In [10]:
# # Creación del archivo Nombre_Output_investigadores


data_2 = cruce1.filter("Similitud >=0.875")\
        .withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id_NIVEL_1"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])

data_3 = data.withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


Nombre_Output_investigadores = data_2.union(data_3)\
                                      .withColumn("Cruce", lit(Ruta_Nombre_Input_investigadores+' / '+Ruta_Nombre_Input_relaciones).cast(StringType()))  

# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Output_investigadores, Flag_csv, Ruta_Nombre_Output_investigadores)


22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 0

22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:26:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 0

In [11]:
# =============================================================================
# Creación del archivo Nombre_Intermedio_cruce1_relaciones
# =============================================================================
Nombre_Intermedio_cruce1_relaciones = data.select(["Id", "IdOld", "Entidad", "Centro","Centro_Norm", "Provincia_Entidad", "Provincia_Centro","CIF","Entidad_Norm"])

In [12]:
 #Eliminamos duplicados
    
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Input_investigadores, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_investigadores, 
                                        "Centro_Norm",
                                        ["Centro_Norm"],
                                        'index_name',
                                        5)

l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l)
duplicados = list(l.filter("final_score >= 0.875").select("Entidad_Norm").dropDuplicates().toPandas().Entidad_Norm)
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.filter(~Nombre_Intermedio_cruce1_relaciones.Entidad_Norm.isin(duplicados))

100%|███████████████████████████████████| 20000/20000 [00:19<00:00, 1017.35it/s]


In [13]:
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.withColumn("Id_NIVEL_1", col("Id"))\
      .withColumn("Id_NIVEL_0", col("IdOld"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_0", col("Entidad"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_1", col("Centro"))\
      .withColumn("COD_PROVINCIA_NIVEL_0", col("Provincia_Entidad"))\
      .withColumn("COD_PROVINCIA_NIVEL_1", col("Provincia_Centro"))\
      .withColumn("NIF_COD_NIVEL_0", col("CIF"))\
      .withColumn("Jerarquia", lit("Orgánica Directa").cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("NIF_COD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_1",lit(None).cast(StringType()))\




Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.select(Nombre_Input_relaciones.columns)

# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Intermedio_cruce1_relaciones, Flag_csv, Ruta_Nombre_Intermedio_cruce1_relaciones)

22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 0

22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:32:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [16]:
# =============================================================================
# Agregacion vertical de relaciones
# =============================================================================

Nombre_Output_relaciones = Nombre_Input_relaciones.union(Nombre_Intermedio_cruce1_relaciones)

# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Output_relaciones, Flag_csv, Ruta_Nombre_Output_relaciones)


if flag_invente_dir3 == True:
# # Nombre_Input_invente
# data invente
    Nombre_Input_invente = pd.read_excel(Ruta_Nombre_Input_invente)
    for i in Nombre_Input_invente.columns:
        Nombre_Input_invente[i] = Nombre_Input_invente[i].astype(str)



    new_header = Nombre_Input_invente.iloc[0] 
    # #grab the first row for the header
    Nombre_Input_invente = Nombre_Input_invente[1:] #take the data less the header row
    Nombre_Input_invente.columns = new_header #set the header row as the df header

    Nombre_Input_invente = Nombre_Input_invente[["DenominacionSocial", "Provincia_Codigo", "NIF",
                                                    "CodigoInvente", "FormaJuridica_Codigo", 
                                                    "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                    "Provincia_Codigo"]]

    Nombre_Input_invente = Nombre_Input_invente.loc[:,~Nombre_Input_invente.columns.duplicated()]

    Nombre_Input_invente = spark.createDataFrame(Nombre_Input_invente.fillna('')) 


    Nombre_Input_invente = Nombre_Input_invente.withColumn("DenominacionSocial_Norm", UDF_normalizarTexto(Nombre_Input_invente.DenominacionSocial))


    # # Nombre_Intermedio_cruce2_invente



    l = get_matching_entities_by_elasticSearch(
                                            Nombre_Intermedio_cruce1_entidades, 
                                            "Entidad_Norm", 
                                            ["Entidad_Norm"],
                                            Nombre_Input_invente, 
                                            "DenominacionSocial_Norm",
                                            Nombre_Input_invente.columns,
                                            'index_name',
                                            1)



    l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")



    l = l.sort(l.final_score.desc()).dropDuplicates(subset = ['Entidad_Norm']).select("Entidad_Norm","DenominacionSocial", "NIF",
                                                                                    "CodigoInvente", "FormaJuridica_Codigo", 
                                                                                    "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                                                    "Provincia_Codigo", "final_score","DenominacionSocial_Norm")



    Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce1_entidades.drop("CodigoInvente","DenominacionSocial", "FormaJuridica_Codigo", 
                                                                       "FormaJuridica_Descripcion", "NIF", "codigoDir3", "codigoOrigen",
                                                                        "Provincia_Codigo", "Provincia_Entidad", "CIF")







    Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente.join(l,
                                          Nombre_Intermedio_cruce2_invente.Entidad_Norm ==  l.Entidad_Norm,"left")                                 



    Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente.filter("final_score is not NULL").drop("Entidad_Norm")
    Nombre_Intermedio_cruce2_invente2 = Nombre_Intermedio_cruce2_invente.filter("final_score is  NULL")




    Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.withColumn("Entidad_Norm", col("DenominacionSocial_Norm"))\
                                                                        .withColumn("Provincia_Entidad", col("Provincia_Codigo"))\
                                                                        .withColumn("CIF", col("NIF"))\
                                                                        .withColumn("Similitud_invente", col("final_score"))




    Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.select(Nombre_Intermedio_cruce1_entidades.columns+["Similitud_invente"])




    Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente1.union(Nombre_Intermedio_cruce2_invente2)

    # =============================================================================
    # Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
    # =============================================================================
    save_csv_parquet(Nombre_Intermedio_cruce2_invente, Flag_csv, Ruta_Nombre_Intermedio_cruce2_invente)



    # # Nombre_Intermedio_proceso_dir3_entidades



    Nombre_Input_dir3 = spark.read.csv(Ruta_Nombre_Input_dir3, sep=";", header=True).drop("Unnamed: 0")
    Nombre_Input_dir3 = Nombre_Input_dir3.withColumn("C_DNM_UD_ORGANICA_Norm", UDF_normalizarTexto(Nombre_Input_dir3.C_DNM_UD_ORGANICA))




    l = get_matching_entities_by_elasticSearch(
                                            Nombre_Intermedio_cruce2_invente, 
                                            "Entidad_Norm", 
                                            ["Entidad_Norm"],
                                            Nombre_Input_dir3, 
                                            "C_DNM_UD_ORGANICA_Norm",
                                            Nombre_Input_dir3.columns,
                                            'index_name',
                                            5)

    l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")\
                                   .dropDuplicates(subset = ['Entidad_Norm', "C_DNM_UD_ORGANICA_Norm"])





    unique = l.selectExpr(
                  '*', 
                  'count(*) over (partition by Entidad_Norm) as cnt'
                ).filter(F.col('cnt') == 1).drop('cnt')




    duplicated = l.join(l.groupBy('Entidad_Norm')\
                      .count().where('count = 1').drop('count'),
                    on=['Entidad_Norm'],
                    how='left_anti')



    duplicated1 = duplicated.selectExpr(
                  '*', 
                  'count(*) over (partition by Entidad_Norm, N_NIVEL_JERARQUICO) as cnt'
                ).filter(F.col('cnt') == 1).drop('cnt')

    duplicated1 = duplicated1.sort(duplicated1.N_NIVEL_JERARQUICO.asc())\
                                .dropDuplicates(subset = ['Entidad_Norm'])



    duplicated2 = l.join(l.groupBy('Entidad_Norm', 'N_NIVEL_JERARQUICO')\
                      .count().where('count = 1').drop('count'),
                    on=['Entidad_Norm','N_NIVEL_JERARQUICO'],
                    how='left_anti')


    duplicated2 = duplicated2.withColumn("Similitud", 
                               udf_Distance_ratcliff_obershelp(
                                   UDF_normalizarTexto(duplicated2.C_DNM_UD_ORGANICA_SUPERIOR),
                                   duplicated2.Entidad_Norm )
                              ) 



    duplicated2 = duplicated2.sort(duplicated2.Similitud.desc())\
                                .dropDuplicates(subset = ['Entidad_Norm']).drop("Similitud")



    match = unique.union(duplicated1).union(duplicated2)\
                   .select(['Entidad_Norm',"final_score"] + Nombre_Input_dir3.columns)\
                   .withColumnRenamed("Entidad_Norm","Entidad_Norm1")



    Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_cruce2_invente\
                                                .drop('C_DNM_UD_ORGANICA_SUPERIOR', 'C_ID_UD_ORGANICA', 'C_ID_TIPO_ENT_PUBLICA',
                                                      'C_DESC_PROV', 'C_ID_DEP_EDP_PRINCIPAL', 'C_ID_ESTADO', 'D_VIG_ALTA_OFICIAL', 
                                                      'CONTACTOS', 'NIF_CIF', 'C_ID_NIVEL_ADMON', 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL', 
                                                      'C_ID_DEP_UD_PRINCIPAL', 'N_NIVEL_JERARQUICO', 'C_DNM_UD_ORGANICA', 
                                                      'C_DNM_UD_ORGANICA_PRINCIPAL', 'B_SW_DEP_EDP_PRINCIPAL', 'C_ID_AMB_PROVINCIA',
                                                      'C_ID_DEP_UD_SUPERIOR')




    Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_proceso_dir3_entidades.join(match,
                                          Nombre_Intermedio_proceso_dir3_entidades.Entidad_Norm ==  match.Entidad_Norm1,"left")     




    Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is not NULL").drop("Entidad_Norm")

    Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is  NULL").drop("Entidad_Norm1")

    Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades2\
                                                    .withColumn("Entidad_Norm", col("C_DNM_UD_ORGANICA"))\
                                                    .withColumn("Provincia_Entidad", col("C_DESC_PROV"))\
                                                    .withColumn("CIF", col("NIF_CIF"))\
                                                    .withColumn("Similitud_Dir3",lit(None).cast(FloatType()))



    Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades1\
                                    .withColumn("Entidad_Norm", col("C_DNM_UD_ORGANICA"))\
                                    .withColumn("Provincia_Entidad", col("C_DESC_PROV"))\
                                    .withColumn("CIF", col("NIF_CIF"))\
                                    .withColumn("Similitud_Dir3", col("final_score"))\
                                    .drop("Entidad_Norm1")




    Nombre_Intermedio_cruce3_dir3 = Nombre_Intermedio_proceso_dir3_entidades1\
                                                .union(Nombre_Intermedio_proceso_dir3_entidades2)\
                                                .drop("C_DNM_UD_ORGANICA_Norm", "final_score")

    # =============================================================================
    # Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
    # =============================================================================
    save_csv_parquet(Nombre_Intermedio_cruce3_dir3, Flag_csv, Ruta_Nombre_Intermedio_cruce3_dir3)


    # # Inclusion vertical



    Nombre_Output_entidades = Nombre_Intermedio_cruce3_dir3.drop("Similitud_Invente", "Similitud_Dir3")




    Nombre_Output_entidades = Nombre_Input_entidades\
                                                .union(Nombre_Output_entidadesdrop("Cruce")
)


    # =============================================================================
    # Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
    # =============================================================================
    save_csv_parquet(Nombre_Output_entidades, Flag_csv, Ruta_Nombre_Output_entidades)


# # If flag_invente_dir3 == False
else:

    Nombre_Output_entidades = Nombre_Input_entidades\
                                                .union(Nombre_Intermedio_cruce1_entidades)

    # =============================================================================
    # Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
    # =============================================================================
    save_csv_parquet(Nombre_Output_entidades, Flag_csv, Ruta_Nombre_Output_entidades)

22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 0

22/08/28 04:45:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/28 04:45:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/28 04:45:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


SniffingError: No viable nodes were discovered on the initial sniff attempt

In [ ]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = True)